# Basic/B2a Example
It is equivalent to the B2a example in Geant4 but re-written with a new more Julia friendly interface. The core of the example is to create an instance of the type `G4JLApplication` with all the essential elements defined (detector and geometry, physics list, primary particles generator, actions, simulated data being accumulated, mapping of logical volumes to sensitive detectors, etc.) 

#### Import the `Geant4` and `Geant4.SystemOfUnits` modules

In [ ]:
using Geant4
using Geant4.SystemOfUnits
using Printf, GeometryBasics

#### Define the Detector Parameters and Geometry
We include a file to define de structure `B2aDetector` and the construction method `B2aConstruct` to make the notebook more manageable

In [ ]:
include(joinpath(@__DIR__, "DetectorB2a.jl"))

#### Define TrackerHit structure

In [ ]:
struct TrackerHit
    trackID::Int32
    chamberNb::Int32
    edep::Float64
    pos::Point3{Float64}
end
function Base.show(io::IO, hit::TrackerHit)
    (;trackID, chamberNb, edep, pos) = hit
    @printf(io, "\ntrackID: %3d chamberNb: %2d Edep: %.3f MeV Position: (%3f, %3f, %3f)", trackID, chamberNb, edep/MeV, pos...) 
end

#### Define Sensitive Detector for the chambers
It implies to define a data structure where to accumulate the TrackerHits for each event, and the callback functions:
- `intilialize` called at the beginning of each event
- `endOfEvent` called at the end of the event
- `processHit`  called for each hit (step ending in the associated logical volume) to be processed

all of the callback functions get the declared data structure as first argument 

In [ ]:
#---SD collected data------------------------------------------------------------------------------
struct B2aSDData <: G4JLSDData
    trackerHits::Vector{TrackerHit}
    B2aSDData() = new([])
end
#---Initialize method------------------------------------------------------------------------------
function _initialize(data::B2aSDData, ::G4HCofThisEvent)::Nothing
    empty!(data.trackerHits)
    return
end
#---End of Event method----------------------------------------------------------------------------
function _endOfEvent(data::B2aSDData, ::G4HCofThisEvent)::Nothing
    return
end
#---Process Hit method-----------------------------------------------------------------------------
function _processHits(data::B2aSDData, step::G4Step, ::G4TouchableHistory)::Bool
    edep = step |> GetTotalEnergyDeposit
    edep <  0. && return false
    pos = step |> GetPostStepPoint |> GetPosition
    push!(data.trackerHits, TrackerHit(step |> GetTrack |> GetTrackID,
                                       step |> GetPreStepPoint |> GetTouchable |> GetCopyNumber,
                                       edep,
                                       Point3{Float64}(x(pos),y(pos),z(pos))))
    return true
end

And create a specialized instance of `G4JLSensitiveDetector` 

In [ ]:
#---Create SD instance-----------------------------------------------------------------------------
chamber_SD = G4JLSensitiveDetector("Chamber_SD", B2aSDData();           # SD name an associated data are mandatory
                                    processhits_method=_processHits,    # process hist method (also mandatory)
                                    initialize_method=_initialize,      # intialize method
                                    endofevent_method=_endOfEvent);     # end of event method


#### Define an End Event Action
This is collect all results each event. Only printing at this time. 

In [ ]:
#---End Event Action-------------------------------------------------------------------------------
function endeventaction(evt::G4Event, app::G4JLApplication)
    hits = getSDdata(app, "Chamber_SD").trackerHits
    eventID = evt |> GetEventID
    if eventID < 10 || eventID % 100 == 0
      G4JL_println("Event: $eventID with $(length(hits)) hits stored in this event")
    end
    return
  end

#### Define the primary particle generator (Particle Gun)

In [ ]:
particlegun = G4JLGunGenerator(particle = "proton", 
                               energy = 3GeV, 
                               direction = G4ThreeVector(0,0,1), 
                               position = G4ThreeVector(0,0,-2940.0))

#### Instantiate a `G4JLApplication` with all the elements
The G4RunManager is created at this moment

In [ ]:
#---Create the Application-------------------------------------------------------------------------
app = G4JLApplication( detector = B2aDetector(nChambers=5),          # detector with parameters
                       generator = particlegun,                      # primary particle generator
                       nthreads = 4,                                 # number of threads (MT)
                       physics_type = FTFP_BERT,                     # what physics list to instantiate
                       endeventaction_method = endeventaction,       # end event action
                       sdetectors = ["Chamber_LV+" => chamber_SD]    # mapping of LVs to SDs (+ means multiple LVs with same name)
                      );             

#### Final steps
Configure the application, initialize the run and fire the beam for 0 events to finalize all initializations

In [ ]:
configure(app)

In [ ]:
initialize(app)

In [ ]:
beamOn(app, 0)

In [ ]:
#---G4Visualization is not yet a registered module------------------------------------------
try
    using G4Visualization
catch e
    using Pkg
    Pkg.develop(path = joinpath(dirname(pathof(Geant4)),"../G4Visualization"))
    using G4Visualization
end
#---Using the CairoMakie backend (static)
using CairoMakie

world = GetWorldVolume()
draw(world)

In [ ]:
beamOn(app,10)

In [ ]:
@time beamOn(app,1000)

In [ ]:
SetParticleEnergy(particlegun, 500MeV)

In [ ]:
@time beamOn(app,1000)